In [1]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import sys





In [2]:



NOTEBOOK_DIR = Path().resolve()


ROOT = NOTEBOOK_DIR.parents[0]


sys.path.append(str(ROOT / "src"))

from bootcamp_data.config import make_paths
from bootcamp_data.io import read_orders_csv, read_users_csv, write_parquet
from bootcamp_data.transforms import enforce_schema

paths = make_paths(ROOT)


In [3]:

Data = ROOT / "data" / "processed" / "analytics_table.parquet"
Figures = ROOT / "reports" / "figures"
Figures.mkdir(parents=True, exist_ok=True)


### Data Frame info

In [4]:
df_data=pd.read_parquet(Data)
print(f"Rows: {len(df_data)}")
print(f"Columns: {len(df_data.columns)}") 
print(df_data.dtypes.head(15))
print(f"missing sum : {df_data.isna().sum().sort_values(ascending=False).head(10)}" )



Rows: 100
Columns: 15
order_id             string[python]
user_id              string[python]
amount                      float64
quantity                    float64
created_at      datetime64[ns, UTC]
status                       object
status_clean                 object
amount _isna                   bool
dates                        object
month                string[python]
year                        float64
day_week                     object
hour                        float64
country                      object
signup_date                  object
dtype: object
missing sum : amount        12
quantity       8
created_at     7
day_week       7
year           7
month          7
dates          7
hour           7
status         0
order_id       0
dtype: int64


In [5]:
print(df_data)

   order_id user_id  amount  quantity                created_at  status  \
0     A0001    0001    12.5       1.0 2025-12-01 10:05:00+00:00    Paid   
1     A0002    0002     8.0       2.0 2025-12-01 11:10:00+00:00    paid   
2     A0003    0003     NaN       1.0 2025-12-02 09:00:00+00:00  Refund   
3     A0004    0001    25.0       NaN 2025-12-03 14:30:00+00:00    PAID   
4     A0005    0004   100.0       1.0                       NaT    paid   
..      ...     ...     ...       ...                       ...     ...   
95    A0096    0069    18.5       1.0 2026-01-19 10:00:00+00:00    paid   
96    A0097    0070    30.0       2.0 2026-01-19 14:45:00+00:00    Paid   
97    A0098    0002     7.0       NaN 2026-01-20 08:15:00+00:00    PAID   
98    A0099    0071    55.0       1.0 2026-01-20 12:00:00+00:00    paid   
99    A0100    0072    12.0       1.0 2026-01-21 10:30:00+00:00    Paid   

   status_clean  amount _isna       dates    month    year   day_week  hour  \
0          paid     

# Qustiones: 


## Q1.Revenue by Country

#### In this section, we explore how revenue is distributed across different countries to understand which markets contribute the most to our business. By aggregating the total revenue per country, we can identify patterns in user spending and assess where our product or service performs best.

In [6]:
revenu_t = (df_data.groupby("country", dropna=False)
.agg(
    n=("order_id","size"),
    revenue=("amount","sum"),
    aov=("amount","mean"),

).reset_index()
)

In [7]:
fig=px.bar(revenu_t ,  x="country",y="revenue",title="Revenue by country")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="country")
fig.update_yaxes(title_text="Revenue(sum amount)")
fig





In [8]:
fig.write_image(str(Figures / "revenue_by_country.png"), scale=2)


#### To start exploring our services across different countries, we analyzed and visualized the total revenue for each country. As shown in the figure above, we can see clear differences in revenue contributions.

#### From the chart, AE appears to generate the lowest revenue compared to SA. This may indicate:

##### -Fewer orders placed in AE,

##### -Lower average order value, or

##### -A smaller sample size that makes the comparison less representative.

#### On the other hand, SA shows the highest revenue, suggesting a larger number of users or higher engagement with our product/service. This also implies that the service provided in SA is performing well, potentially leading to repeat orders and new customer acquisition.


## Q2.Revenue Trend Across Dates


#### To understand how revenue changes over time, we analyzed the trend of revenue across different dates in the dataset. This helps identify peaks and dips in orders and provides insights into user behavior and engagement over time.

In [9]:
rev_trnd=(df_data.groupby("dates", dropna=False).agg(
    n=("order_id","size") , revenue=("amount","sum")).reset_index()
)

In [10]:
fig=px.line(rev_trnd ,x="dates", y="revenue",title="Revenue Over Dates")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Revenue")
fig





In [11]:
fig.write_image(str(Figures / "revenue_trend_dates.png"), scale=2)


#### By visualizing this trend, we can better plan marketing, promotions, and service improvements, and observe how users interact with our product throughout different periods.

## Q3.Order Amount Distribution

#### To understand customer spending behavior, we analyzed the distribution of order amounts in the dataset. This visualization shows how frequently different order sizes occur, highlights typical spending ranges, and helps identify outliers or unusually high/low orders.


In [12]:
if "amount_winsor" not in df_data.columns and "amount" in df_data.columns:
    lo,hi = df_data["amount"].quantile([0.01, 0.99])
    df_data["amount_winsor"] = df_data["amount"].clip(lo, hi)

In [13]:
fig=px.histogram(df_data ,x="amount_winsor",nbins=30 ,title="Order amount disribution ")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
fig





In [14]:
fig.write_image(str(Figures / "Amount_distribution.png"), scale=2)


#### The histogram above shows the distribution of order amounts after winsorization. The x-axis represents the order amount, and the y-axis shows the number of orders. We can see a peak in the 10–20 amount range, with a count of 31 orders, indicating that most customers place orders within this typical spending range. The tails represent outliers or unusually large/small orders, which help us understand spending behavior, detect anomalies, and inform pricing or promotional strategies

## Q4.Order Status Analysis (Paid vs Refunded)


#### In this section, we analyze the distribution of order statuses to understand which status is more prevalent and what insights we can gain about our service or product. Based on the data, paid orders clearly dominate over refunded orders, which suggests that the majority of users successfully complete their purchases.

In [15]:
df_data

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount _isna,dates,month,year,day_week,hour,country,signup_date,amount_winsor
0,A0001,0001,12.5,1.0,2025-12-01 10:05:00+00:00,Paid,paid,False,2025-12-01,2025-12,2025.0,Monday,10.0,SA,2025-11-15,12.5
1,A0002,0002,8.0,2.0,2025-12-01 11:10:00+00:00,paid,paid,False,2025-12-01,2025-12,2025.0,Monday,11.0,SA,2025-11-20,8.0
2,A0003,0003,NaN,1.0,2025-12-02 09:00:00+00:00,Refund,refund,False,2025-12-02,2025-12,2025.0,Tuesday,9.0,AE,2025-11-22,NaN
3,A0004,0001,25.0,NaN,2025-12-03 14:30:00+00:00,PAID,paid,False,2025-12-03,2025-12,2025.0,Wednesday,14.0,SA,2025-11-15,25.0
4,A0005,0004,100.0,1.0,NaT,paid,paid,False,None,<NA>,NaN,None,NaN,SA,2025-11-25,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A0096,0069,18.5,1.0,2026-01-19 10:00:00+00:00,paid,paid,False,2026-01-19,2026-01,2026.0,Monday,10.0,SA,2025-11-24,18.5
96,A0097,0070,30.0,2.0,2026-01-19 14:45:00+00:00,Paid,paid,False,2026-01-19,2026-01,2026.0,Monday,14.0,SA,2025-11-25,30.0
97,A0098,0002,7.0,NaN,2026-01-20 08:15:00+00:00,PAID,paid,False,2026-01-20,2026-01,2026.0,Tuesday,8.0,SA,2025-11-20,7.0
98,A0099,0071,55.0,1.0,2026-01-20 12:00:00+00:00,paid,paid,False,2026-01-20,2026-01,2026.0,Tuesday,12.0,AE,2025-11-26,55.0


In [16]:
orders_info=(df_data.groupby("status_clean", dropna=False).agg(
    n=("order_id","count") ,
).reset_index())

In [17]:
fig=px.bar(orders_info, x="status_clean", y="n" ,title="Order Status Analysis")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Order Status")
fig.update_yaxes(title_text="Number of orders")
fig


In [18]:
fig.write_image(str(Figures / "Order_Status_analysis.png"), scale=2)


#### After grouping and cleaning the order status data, we visualized the results to interpret the trends. As shown in the figure above, the number of orders with a paid status is significantly higher than those marked as refund. Specifically, there are around 80 paid orders compared to 20 refunded orders.

This large difference provides valuable insight into user behavior. The small percentage of refunded orders could be due to users changing their minds, minor system errors, or product-related issues. In other words, some refunds may occur simply because users forgot about their orders.

Overall, the higher number of paid orders indicates strong user engagement and satisfaction with the product and service. It suggests that users are completing their purchases and even potentially reordering, which is a positive sign for the overall user experience

## 4.1 Bootstrap Comparison in Order Refund Orders Between SA and AE

##### After visualizing the distribution of orders by status, we can clearly see that paid orders significantly outnumber refunded orders. This gives us a first intuitive understanding: most users complete their purchases successfully, while refunds remain a small portion of the total orders.

##### To dive deeper into potential differences in user behavior across countries, we performed a bootstrap comparison of refund rates between Saudi Arabia (SA) and United Arab Emirates (AE). By resampling the data 2000 times, we estimated the mean difference in refund rates and computed a 95% confidence interval.


In [19]:
import numpy as np


def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"
    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)
    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

d = df_data.assign(is_refund=df_data["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))

result = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(result)


n_SA: 76 n_AE: 24
{'diff_mean': -0.0899122807017544, 'ci_low': -0.2807017543859649, 'ci_high': 0.07461622807017514}


#### The results show that the difference in refund rates (diff_mean) is slightly negative, indicating that AE users tend to have a marginally higher refund rate than SA users. However, the confidence interval crosses zero, suggesting that this difference is not statistically significant.

### **Findings:**

- **Revenue by country**:  
  - Saudi Arabia (SA) generates the highest revenue, suggesting more users and higher engagement.  
  - United Arab Emirates (AE) generates the lowest revenue, likely due to fewer orders, lower average order value, or smaller sample size.
- **Revenue trend over time**: Peaks and dips observed across the dataset, indicating seasonal or periodic fluctuations in user activity.  
- **Amount distribution**: Most orders fall within the $10–20 range (31 orders), with outliers representing unusually high or low order amounts.  
- **Paid vs refunded orders**: 80 paid orders vs 20 refunded orders. Paid orders dominate, showing strong user engagement and satisfaction.  
- **Refund rate by country**: Bootstrap comparison between SA and AE shows AE has a slightly higher refund rate, but the difference is not statistically significant.